In [1]:
import numpy as np
import  matplotlib.pyplot as plt

# board dimension
LENGTH = 3

In [2]:
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1 #  represents 'x' on the board, player 1
        self.o = 1  # represnets 'o' on the board, player 2      
        self.winner = None
        self.ended = False
        self.num_states = 3 ** (LENGTH * LENGTH)
        
        
    def is_empty(self, i, j):
        return self.board[i, j] == 0
    
    
    def reward(self, symbol):
        # no reward if the game is not over
        if not self.game_over():
            return 0
        
        # game is over 
        if self.winner == symbol:
            return 1
        else:
            return 0

    
    def get_state(self):
        # returns the current state as an int represented by the base 3 number (0, x, o)
        k = 0
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i,j] == self.x:
                    v = 1
                elif self.board[i,j] == self.o:
                    v = 2
                h+= (3**k)* v
                k+= 1
        return h
    
  

    def game_over(self, force_recalculate=False):
        if not force_recalculate and self.ended:
            return self.ended

        # check rows
        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
        
        # check colums
        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
                
        # check diagonals
        for player in (self.x, self.o):
            # top right -> bottom left
            if self.board.trace() == player*(LENGTH):
                self.winner = player
                self.ended = True
                return True
            
            if np.fliplr(self.board).trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True
            
        # check if draw
        if np.all((self.board == 0) == False):
            self.winner = None
            self.ended = True
            return True
        
        # game is not over
        self.winner = None
        return False
    
    def is_draw(self):
        return self.winner is None and self.ended 
                
        
    def draw_board(self):
        for i in range(LENGTH):
          print("-------------")
          for j in range(LENGTH):
            print("  ", end="")
            if self.board[i,j] == self.x:
              print("x ", end="")
            elif self.board[i,j] == self.o:
              print("o ", end="")
            else:
              print("  ", end="")
          print("")
        print("-------------")    

In [3]:
class Agent:
    def __init__(self, eps=0.1, alpha=0.5):
        # probability of choosing random action instead of greedy
        self.eps = eps
        self.alpha = alpha
        self.verbose = False
        self.state_history = []
        
    def setV(self, v):
        self.V = v
        
    def set_sym(self, symbol):
        self.sym = symbol
    
    def set_verbose(self, b):
        self.verbose = b
        
    def reset_history(self):
        self.state_history = []
        
    def take_action(self, env):
        
        # epsilon greedy strategy
        r = np.random.rand()
        best_state = None
        
        if r < self.eps :
            # taking random action
            if self.verbose:
                print("taking random action")
            
            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i,j))
            index = np.random.choice(len(possible_moves))
            next_move = possible_moves[index]
            
        else:
            # choose the next move based on the current values of the states
            position_to_value = {}
            next_move = None
            best_value = -1
            
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        # what's the state if we made this move
                        env.board[i,j] = self.sym
                        state = env.get_state()
                        
                        # set the state back to 0
                        env.board[i,j] = 0
                        position_to_value[(i, j )] = self.V[state]
                        
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i, j)
            
            if self.verbose :
                print("taking a greedy action")
                for i in range(LENGTH):
                    print("-----------------")
                    for j in range(LENGTH):
                        if env.is_empty(i, j):
                            print(" %.2f|" % position_to_value[(i, j)], end="")
                        else:
                            print(" ", end="")
                            if env.board[i,j] == env.x :
                                print("x |", end="")
                            elif env.board[i,j] == env.o:
                                print("o |", end="")
                            else:
                                print(" |", end="")
                                    
                    print("")
                print("-----------------")
                            
        # make the move
        env.board[next_move[0], next_move[1]] = self.sym
            
                
    def update_state_history(self, s):
        self.state_history.append(s)
        
    def update(self,env):
        # V(previous_state) =  V(previous_state) + alpha * (V(next_state) - V(previous_state))
        reward = env.reward(self.sym)
        target = reward
        
        for previous in reversed(self.state_history):
            value = self.V[previous] + self.alpha*( target - self.V[previous])
            self.V[previous] = value
            target = value
            
        self.reset_history()

In [4]:
class Human:
    def __init__(self):
        pass
    
    def set_sym(self, symbol):
        self.sym = symbol
        
    def take_action(self, env):
        while(True):
            move = input("enter co-ordinates i,j=0..2 for your next move")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            
            if env.is_empty(i, j):
                env.board[i,j] = self.sym
                break
                
    def update(self, env):
        pass
    
    def update_state_history(self, s):
        pass
            
            

In [5]:
def get_state_hash_winner(env, i = 0, j = 0):
    results = []
    
    for v in (0, env.x, env.o):
        env.board[i,j] = v
        
        if j == 2:
            # set j to 0 , unless i = 2, in which case we are done
            if i == 2 :
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                results.append((state, winner, ended))
                
            else:
                results += get_state_hash_winner(env, (i+1), 0)
        else:
            results += get_state_hash_winner(env, i, j+1)
            
    return results               

In [6]:
def initialV_x(env, state_winner_triples):
    V = np.zeros(env.num_states)
    
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v =1 
            else:
                v= 0
        else:
            v = 0.5
        V[state] = v
    return V

In [7]:
def initialV_o(env, state_winner_triples):
    V = np.zeros(env.num_states)
    
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v =1 
            else:
                v= 0
        else:
            v = 0.5
        V[state] = v
    return V

In [8]:
def play_game(p1, p2, env, draw=False):
    current_player = None
    
    while not env.game_over():
        # let p1 start first
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
            
        # draw the board       
        if draw:
            if draw == 1 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()
                
        # current player makes a move
        current_player.take_action(env)
        
        # update state histories
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
        
        
    if draw:
        env.draw_board()
        
    #update value function
    p1.update(env)
    p2.update(env)

In [ ]:
if __name__ == '__main__':
    p1 = Agent()
    p2 = Agent()
    
    env = Environment()
    state_winner_triples = get_state_hash_winner(env)
    
    Vx = initialV_x(env, state_winner_triples)
    p1.setV(Vx)
    
    Vo = initialV_o(env, state_winner_triples)
    p2.setV(Vo)
    
    p1.set_sym(env.x)
    p2.set_sym(env.o)
    
    iterations = 10000
    for t in range(iterations):
        if t %1000 == 0:
            print(t)
        play_game(p1, p2, Environment())
        
    
    # human vs agent
    human = Human()
    human.set_sym(env.o)
    
    while(True):
        p1.set_verbose(True)
        # let's make agent player 1 and see if it pick center as the starting position
        play_game(p1,human, Environment(), draw=2)
        
        answer =input("play again [Y/n]?") 
        if answer and answer.lower()[0] == 'n':
            break    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
taking a greedy action
-----------------
 0.49| 0.66| 0.54|
-----------------
 0.55| 0.97| 0.54|
-----------------
 0.50| 0.63| 0.78|
-----------------
-------------
            
-------------
      x     
-------------
            
-------------
enter co-ordinates i,j=0..2 for your next move0,0
taking a greedy action
-----------------
 o | 0.81| 0.32|
-----------------
 0.80| x | 0.59|
-----------------
 0.97| 0.68| 0.69|
-----------------
-------------
  o         
-------------
      x     
-------------
  x         
-------------
enter co-ordinates i,j=0..2 for your next move0,2
taking a greedy action
-----------------
 o | 0.93| o |
-----------------
 0.03| x | 0.03|
-----------------
 x | 0.12| 0.00|
-----------------
-------------
  o   x   o 
-------------
      x     
-------------
  x         
-------------
enter co-ordinates i,j=0..2 for your next move2,1
taking a greedy action
-----------------
 o | x | o |
-----------------
 0